In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.neural_network import BernoulliRBM
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

CODE TO LOAD MIN-MAX POOLED MFCC FILES that are stored in a particular folder

In [8]:
def load_mfcc_files(directory):
    mfcc_data = []
    for file in os.listdir(directory):
        if file.endswith('-MFCC.csv'):
            file_path = os.path.join(directory, file)
            data = pd.read_csv(file_path, header=None).values
            mfcc_data.append(data)
    return mfcc_data


code for scalling and building the cdbn model(subject to hyper-parameter tunin)

In [13]:
def preprocess_mfcc(mfccs):
    """
    Preprocess MFCC features.
    
    Parameters:
    mfccs (numpy.ndarray): The MFCC features.
    
    Returns:
    numpy.ndarray: The preprocessed MFCC features.
    """
    scaler = StandardScaler()
    mfccs_scaled = scaler.fit_transform(mfccs)
    mfccs_scaled = mfccs_scaled[np.newaxis, ..., np.newaxis]  # Add batch and channel dimensions
    return mfccs_scaled

In [14]:

class RBMLayer(layers.Layer):
    def __init__(self, n_components, learning_rate=0.01, batch_size=10, n_iter=10, **kwargs):
        super(RBMLayer, self).__init__(**kwargs)
        self.rbm = BernoulliRBM(n_components=n_components, learning_rate=learning_rate, batch_size=batch_size, n_iter=n_iter)
    
    def call(self, inputs):
        return self.rbm.transform(inputs)
    
    def fit(self, X):
        self.rbm.fit(X)

def build_cdbn(input_shape, conv_rbm_layers, dense_layers):
    """
    Build a Convolutional Deep Belief Network (CDBN).
    
    Parameters:
    input_shape (tuple): Shape of the input data.
    conv_rbm_layers (list of tuples): List of convolutional RBM layers, each defined by (filters, kernel_size, pool_size, n_components).
    dense_layers (list of int): List of dense layer sizes.
    
    Returns:
    tensorflow.keras.Model: The CDBN model.
    """
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=input_shape))
    
    # Add convolutional RBM layers
    for filters, kernel_size, pool_size, n_components in conv_rbm_layers:
        model.add(layers.Conv2D(filters, kernel_size, activation='relu'))
        model.add(layers.MaxPooling2D(pool_size))
        model.add(RBMLayer(n_components=n_components))
    
    model.add(layers.Flatten())
    
    # Add dense layers
    for units in dense_layers:
        model.add(layers.Dense(units, activation='relu'))
    
    model.add(layers.Dense(6, activation='softmax'))  # Adjust the number of classes as needed
    return model

# Example usage
input_shape = (40, 173, 1)  # Example shape for MFCC (e.g., 40 MFCCs over 173 frames)
conv_rbm_layers = [
    (32, (3, 3), (2, 2), 64),
    (64, (3, 3), (2, 2), 128),
    (128, (3, 3), (2, 2), 256)
]
dense_layers = [128]

cdbn_model = build_cdbn(input_shape, conv_rbm_layers, dense_layers)
cdbn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

c:\Users\kani1\OneDrive\Documents\DS203 Programing for Data Science\venv\Lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


NameError: name 'BernoulliRBM' is not defined

code to visualize what cdbn is doing (subject to change)

In [10]:
def visualize_feature_maps(model, layer_name, input_data):
    """
    Visualize the feature maps of a convolutional layer.
    
    Parameters:
    model (tensorflow.keras.Model): The trained model.
    layer_name (str): The name of the convolutional layer.
    input_data (numpy.ndarray): The input data.
    """
    layer = model.get_layer(name=layer_name)
    feature_map_model = models.Model(inputs=model.input, outputs=layer.output)
    feature_maps = feature_map_model.predict(input_data)
    
    n_features = feature_maps.shape[-1]
    fig, axes = plt.subplots(1, n_features, figsize=(20, 5))
    
    for i in range(n_features):
        axes[i].imshow(feature_maps[0, :, :, i], cmap='viridis')
        axes[i].axis('off')
    
    plt.show()

def visualize_filters(model, layer_name):
    """
    Visualize the filters of a convolutional layer.
    
    Parameters:
    model (tensorflow.keras.Model): The trained model.
    layer_name (str): The name of the convolutional layer.
    """
    layer = model.get_layer(name=layer_name)
    filters, biases = layer.get_weights()
    
    # Normalize filter values to 0-1 so we can visualize them
    f_min, f_max = filters.min(), filters.max()
    filters = (filters - f_min) / (f_max - f_min)
    
    n_filters = filters.shape[-1]
    fig, axes = plt.subplots(1, n_filters, figsize=(20, 5))
    
    for i in range(n_filters):
        f = filters[:, :, :, i]
        axes[i].imshow(f[:, :, 0], cmap='viridis')
        axes[i].axis('off')
    
    plt.show()

In [12]:

directory = "C:\\Users\\kani1\\OneDrive\\Documents\\DS203 Programing for Data Science\\project\\mfcc_processed" #path of the folder which contains the min-max pooled mfcc files
mfcc_predict_data = load_mfcc_files(directory)

mfcc_predict_data_combined = np.array(mfcc_predict_data)

# Print the shape of the combined numpy array
print(mfcc_predict_data_combined.shape)


(116, 20, 25000)


In [ ]:
mfccs_preprocessed = preprocess_mfcc(mfccs_data_combined)
input_shape = mfccs_preprocessed.shape[1:]
cdbn_model = build_cdbn(input_shape)
cdbn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


X_train = np.array([0])  # Replace with actual data
y_train = np.array([0])  # Replace with actual labels

# Train the model
cdbn_model.fit(X_train, y_train, epochs=10, batch_size=1)

visualize_filters(cdbn_model, 'conv2d')
visualize_feature_maps(cdbn_model, 'conv2d', mfccs_preprocessed)

# Predict on new data
X_test = np.array([0])  # Replace with actual test data
predictions = cdbn_model.predict(X_test)
print(predictions)